In [1]:
from subgrounds.subgrounds import Subgrounds
from subgrounds.plotly_wrappers import Bar, Figure

import pandas as pd
import plotly.express as px


# initialize Subgrounds object
sg = Subgrounds()

# load livepeer subgraph
lpt = sg.load_subgraph('https://api.thegraph.com/subgraphs/name/livepeer/livepeer')

# define FieldPaths
transcoders = lpt.Query.transcoders
delegators = lpt.Query.delegators

In [3]:
top_10_delegators = delegators(
    orderBy='delegatedAmount',
    orderDirection='desc',
    first=10
)

sg.query_df([
    top_10_delegators.id,
    top_10_delegators.delegatedAmount,
    top_10_delegators.delegate.id
])

,delegators_id,delegators_delegatedAmount,delegators_delegate_id
0,0x9c10672cee058fd658103d90872fe431bb6c0afa,2.516184e+06,0x9c10672cee058fd658103d90872fe431bb6c0afa
1,0xda43d85b8d419a9c51bbf0089c9bd5169c23f2f9,1.252157e+06,0xda43d85b8d419a9c51bbf0089c9bd5169c23f2f9
2,0x525419ff5707190389bfb5c87c375d710f5fcb0e,9.233017e+05,0x525419ff5707190389bfb5c87c375d710f5fcb0e
3,0x4ff088ac5422f994486663ff903b040692797168,7.841820e+05,0x4ff088ac5422f994486663ff903b040692797168
4,0x942f0c28fb85ea0b50bfb76a3ecfa99861fa9b4b,7.284690e+05,0x942f0c28fb85ea0b50bfb76a3ecfa99861fa9b4b
5,0xe0a4a877cd0a07da7c08dffebc2546a4713147f2,6.424943e+05,0xe0a4a877cd0a07da7c08dffebc2546a4713147f2
6,0x4f4758f7167b18e1f5b3c1a7575e3eb584894dbc,5.467493e+05,0x4f4758f7167b18e1f5b3c1a7575e3eb584894dbc
7,0x731808ad8b1c3d13e8972db838ada5fc6ae3c2c8,5.117972e+05,0x731808ad8b1c3d13e8972db838ada5fc6ae3c2c8
8,0xd84781e1a9b74d71ea76cda8bb9f30893bfd00d1,4.195702e+05,0xd84781e1a9b74d71ea76cda8bb9f30893bfd00d1
9,0xdac817294c0c87ca4fa1895ef4b972eade99f2fd,3.410711e+05,0xdac817294c0c87ca4fa1895ef4b972eade99f2fd


hmm, this matches the total stake leaderboard 1:1

In [4]:
top_10_transcoders = transcoders(
    orderBy=transcoders.totalStake,
    orderDirection='desc',
    first=10
)

sg.query_df([
    transcoders.id,
    transcoders.delegators.id
])

,transcoders_id,transcoders_delegators_id
0,0x9c10672cee058fd658103d90872fe431bb6c0afa,0x005839abcf30fc149dbac154d62cf00ce5a18351
1,0x9c10672cee058fd658103d90872fe431bb6c0afa,0x00bace4de010c42f0452fe27fce695aee89ab19d
2,0x9c10672cee058fd658103d90872fe431bb6c0afa,0x0354845b1a7ceac8a2343edb66303cef5211c5cd
3,0x9c10672cee058fd658103d90872fe431bb6c0afa,0x04288d0b8bc6298a7cc26f367f42932d71c79d9a
4,0x9c10672cee058fd658103d90872fe431bb6c0afa,0x05c2974d048d90ec5bfa124b6369396727b900fd
...,...,...
696,0xdac817294c0c87ca4fa1895ef4b972eade99f2fd,0xeff76f578f5b159164907f8dae5ea03d28a540f5
697,0xdac817294c0c87ca4fa1895ef4b972eade99f2fd,0xf4421cdaf6df0a0efa85f596369c6f1115e63659
698,0xdac817294c0c87ca4fa1895ef4b972eade99f2fd,0xf61f1bb7fcba897750b521bf4deab314eab1c70c
699,0xdac817294c0c87ca4fa1895ef4b972eade99f2fd,0xf91bcd7f52d76e8f59c0f67694b269e9a863d124


In [5]:
top_by_delegators = sg.query_df([
    transcoders.id,
    transcoders.delegators.id
])

top_by_delegators.groupby('transcoders_id').count()

,transcoders_delegators_id
transcoders_id,
0x4f4758f7167b18e1f5b3c1a7575e3eb584894dbc,30
0x4ff088ac5422f994486663ff903b040692797168,100
0x525419ff5707190389bfb5c87c375d710f5fcb0e,100
0x731808ad8b1c3d13e8972db838ada5fc6ae3c2c8,2
0x942f0c28fb85ea0b50bfb76a3ecfa99861fa9b4b,14
0x9c10672cee058fd658103d90872fe431bb6c0afa,100
0xd84781e1a9b74d71ea76cda8bb9f30893bfd00d1,100
0xda43d85b8d419a9c51bbf0089c9bd5169c23f2f9,100
0xdac817294c0c87ca4fa1895ef4b972eade99f2fd,55


looks like there might be a limit at 100...

In [6]:
delegators_unlimited = transcoders.delegators(
    first=500
)

top_by_delegators_2 = sg.query_df([
    transcoders.id,
    delegators_unlimited.id
])

top_by_delegators_2.groupby('transcoders_id').count()

,transcoders_delegators_id
transcoders_id,
0x4f4758f7167b18e1f5b3c1a7575e3eb584894dbc,30
0x4ff088ac5422f994486663ff903b040692797168,119
0x525419ff5707190389bfb5c87c375d710f5fcb0e,500
0x731808ad8b1c3d13e8972db838ada5fc6ae3c2c8,2
0x942f0c28fb85ea0b50bfb76a3ecfa99861fa9b4b,14
0x9c10672cee058fd658103d90872fe431bb6c0afa,193
0xd84781e1a9b74d71ea76cda8bb9f30893bfd00d1,155
0xda43d85b8d419a9c51bbf0089c9bd5169c23f2f9,263
0xdac817294c0c87ca4fa1895ef4b972eade99f2fd,55


yep - #3 has a lot. next run might take a bit...

In [8]:
delegators_really_unlimited = transcoders.delegators(
    first=5000
)

top_by_delegators_3 = sg.query_df([
    transcoders.id,
    delegators_really_unlimited.id
])

top_by_delegators_3.groupby('transcoders_id').count()

,transcoders_delegators_id
transcoders_id,
0x4f4758f7167b18e1f5b3c1a7575e3eb584894dbc,30
0x4ff088ac5422f994486663ff903b040692797168,119
0x525419ff5707190389bfb5c87c375d710f5fcb0e,716
0x731808ad8b1c3d13e8972db838ada5fc6ae3c2c8,2
0x942f0c28fb85ea0b50bfb76a3ecfa99861fa9b4b,14
0x9c10672cee058fd658103d90872fe431bb6c0afa,193
0xd84781e1a9b74d71ea76cda8bb9f30893bfd00d1,155
0xda43d85b8d419a9c51bbf0089c9bd5169c23f2f9,263
0xdac817294c0c87ca4fa1895ef4b972eade99f2fd,55


max of top 10 is 716. expand to 100 transcoders w max delegators 750

In [9]:
top_10_transcoders = transcoders(
    orderBy=transcoders.totalStake,
    orderDirection='desc',
    first=100
)

delegators_750 = transcoders.delegators(
    first=750
)

top_by_delegators_4 = sg.query_df([
    transcoders.id,
    delegators_750.id
])

top_by_delegators_4.groupby('transcoders_id').count()

,transcoders_delegators_id
transcoders_id,
0x00803b76dc924ceabf4380a6f9edc2ddd3c90f38,2
0x02b6aac33a397aaadee5227c70c69bb97f2cc529,2
0x0d5a6601942bf92e7b259bfc6b56009e6a1c0ad2,1
0x0fc80afb7876f579f1fb1c4d1c37cf1339038658,1
0x104a7ca059a35fd4def5ecb16600b2caa1fe1361,1
...,...
0xf5a88945ce64b5648b895e05eb1edf693dde8905,2
0xf63b84aaa39c4406ebf2d1e3d6acb1fb51f19dbd,2
0xf8ee1a8af53086b025dcc9b74a2444827d3ec64c,1


In [14]:
top_100 = top_by_delegators_4.groupby('transcoders_id').count()
top_100.sort_values(by=['transcoders_delegators_id'], ascending=False)

,transcoders_delegators_id
transcoders_id,
0x525419ff5707190389bfb5c87c375d710f5fcb0e,716
0xe9e284277648fcdb09b8efc1832c73c09b5ecf59,452
0xda43d85b8d419a9c51bbf0089c9bd5169c23f2f9,263
0x9c10672cee058fd658103d90872fe431bb6c0afa,193
0xd18a02647d99dc9f79afbe0f58f8353178e6141f,183
...,...
0xa7c3be4eb0268dcbefbd30d8f7ac8f1a57468032,1
0x84939cbda5c5a83e0f0a23204953e105a31de584,1
0x9e48d670d2bd7300796caa6c05e3d2cc41b8cb9c,1
